<a href="https://colab.research.google.com/github/kishorepv/search/blob/main/4_AI_Introduction_to_Lexical_and_BM25_Is_there_a_better_TFIDF%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lexical tokenization - Better TF\*IDF

Let's walk through a basic introduction to lexical search.

### Who you are:

An ML engineer with enough comfort with Python data stack (pandas, numpy, etc) that wants to understand traditional search engines (ie Elasticsearch, etc)

### What this is

A run through of the core concepts behind lexical search.


## This notebook: Better TFIDF (hint maybe its name begins with B)?

We [previously walked through text similarity scoring](https://colab.research.google.com/drive/1MOUa7u6kE_BWJEeueWjRuClctVxaJlJL#scrollTo=UevYFMMZmbp9). We used a first pass TF\*IDF formulation. But we'll see how that has problems, and how it can be improved.

In [1]:
!pip install searcharray pystemmer

from searcharray import SearchArray
import pandas as pd
import numpy as np
import Stemmer


## Tokenize and index

Tokenize and index two fields:

1. The name (who's chatting)
2. Their message

**Note this time** we've added some longer messages

In [2]:
from string import punctuation
stemmer = Stemmer.Stemmer('english')


def even_better_tokenize(text):
    lowercased = text.lower()
    without_punctuation = lowercased.translate(str.maketrans('', '', punctuation))
    split = without_punctuation.split()
    return [stemmer.stemWord(tok) for tok in split]


chat_transcript = [
  "Hi this is Doug, I have a complaint about the weather. My Doug Day is not Doug-tastic.",

  """
    Doug, we see you're having an issue with the climate. Doug, maybe you'd like to talk to the manager?
    Doug I think that'd be wise. What do you think Doug?
  """,
  "Tom, can I speak to your manager?",
  "Hi, this is Sue, Tom's boss. What can I do for you?",
  "I have complaints about the ski conditions in West Virginia",
  "Oh doug thats terrible, lets see what we can do."
]

msgs = pd.DataFrame({"name": ["Doug", "Tom", "Doug", "Sue", "Doug", "Sue"],
                     "msg": chat_transcript})
msgs['msg_tokenized'] = SearchArray.index(msgs['msg'],
                                          tokenizer=even_better_tokenize)
msgs['name_tokenized'] = SearchArray.index(msgs['name'],
                                          tokenizer=even_better_tokenize)
msgs

2025-09-22 08:26:16,575 - searcharray.indexing - INFO - Indexing begins w/ 4 workers


INFO:searcharray.indexing:Indexing begins w/ 4 workers


2025-09-22 08:26:16,579 - searcharray.indexing - INFO - 0 Batch Start tokenization


INFO:searcharray.indexing:0 Batch Start tokenization


2025-09-22 08:26:16,582 - searcharray.indexing - INFO - Tokenizing 6 documents


...

2025-09-22 08:26:16,636 - searcharray.indexing - INFO - (main thread) Processing 1 batch results


INFO:searcharray.indexing:(main thread) Processing 1 batch results


2025-09-22 08:26:16,638 - searcharray.indexing - INFO - Indexing from tokenization complete


INFO:searcharray.indexing:Indexing from tokenization complete


,name,msg,msg_tokenized,name_tokenized
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'is', 'a', 'day', 'not', 'doug', ...",Terms({'doug'})
1,Tom,"\n Doug, we see you're having an issue with...","Terms({'doug', 'wise', 'an', 'what', 'talk', '...",Terms({'tom'})
2,Doug,"Tom, can I speak to your manager?","Terms({'your', 'can', 'tom', 'to', 'speak', 'i...",Terms({'doug'})
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'hi', 'what', 'is', 'boss', 'sue', 'do'...",Terms({'sue'})
4,Doug,I have complaints about the ski conditions in ...,"Terms({'in', 'west', 'about', 'condit', 'virgi...",Terms({'doug'})
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'what', 'doug', 'see', 'do', 'we', 'tha...",Terms({'sue'})


## Search again w/ TF\*IDF

Recall we created a naive TF\*IDF similarity function last time. Let's use that!

In [3]:
from searcharray.similarity import Similarity

def tf_idf(term_freqs: np.ndarray,        # TF array of every doc in the index
               doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
               doc_lens: np.ndarray,          # Every documents length (same shape as TF)
               avg_doc_lens: int,             # avg doc length of corpus
               num_docs: int) -> np.ndarray:     # total number of docs in corpus

    return term_freqs / (doc_freqs + 1)


In [4]:
QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

# ACCUMULATE SCORES
scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    # PASS SIMILARITY
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf)
        # Take maximum between field_scores and this field's score
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [0.5  1.   0.   0.   0.   0.25]
Field name_tokenized, Term 'doug' score: [0.25 0.   0.25 0.   0.25 0.  ]
Term 'doug' score: [0.5  1.   0.25 0.   0.25 0.25]
Scores now: [0.5  1.   0.25 0.   0.25 0.25]
Field msg_tokenized, Term 'complaint' score: [0.33333333 0.         0.         0.         0.33333333 0.        ]
Field name_tokenized, Term 'complaint' score: [0. 0. 0. 0. 0. 0.]
Term 'complaint' score: [0.33333333 0.         0.         0.         0.33333333 0.        ]
Scores now: [0.33333333 0.         0.         0.         0.33333333 0.        ]


,name,msg,msg_tokenized,name_tokenized,scores
1,Tom,"\n Doug, we see you're having an issue with...","Terms({'doug', 'wise', 'an', 'what', 'talk', '...",Terms({'tom'}),1.000000
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'is', 'a', 'day', 'not', 'doug', ...",Terms({'doug'}),0.833333
4,Doug,I have complaints about the ski conditions in ...,"Terms({'in', 'west', 'about', 'condit', 'virgi...",Terms({'doug'}),0.583333
2,Doug,"Tom, can I speak to your manager?","Terms({'your', 'can', 'tom', 'to', 'speak', 'i...",Terms({'doug'}),0.250000
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'what', 'doug', 'see', 'do', 'we', 'tha...",Terms({'sue'}),0.250000
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'hi', 'what', 'is', 'boss', 'sue', 'do'...",Terms({'sue'}),0.000000


## Problem - Term frequency saturation

**Problem** - we notice this one document that says "Doug" a lot gets a high score. Obviously it has a high term frequency, but that doesn't automatically mean its more relevant _to our users_

Early Information Researchers realized, we don't want to take _raw_ term frequency, because after a while, it doesn't 'add' signal to the score. The core concept is **aboutness** - is this document _about_ the term? After a point saying "doug" over and over doesn't make it any more 'about' Doug.



### Take log of term freq?

Let's try a simple saturation, taking the log of term freq.

In [5]:
from searcharray.similarity import Similarity

def tf_idf_saturate(term_freqs: np.ndarray,        # TF array of every doc in the index
                    doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
                    doc_lens: np.ndarray,          # Every documents length (same shape as TF)
                    avg_doc_lens: int,             # avg doc length of corpus
                    num_docs: int) -> np.ndarray:     # total number of docs in corpus

    # the "+ 1e-4" within the log is to avoid log 0 issue
    return np.log(term_freqs + 1e-4) / (doc_freqs + 1) # TAKE LOG




QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf_saturate) # CHANGED
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [ 1.73299283e-01  3.46579850e-01 -2.30258512e+00 -2.30258512e+00
 -2.30258512e+00  2.50028988e-05]
Field name_tokenized, Term 'doug' score: [ 2.50028988e-05 -2.30258512e+00  2.50028988e-05 -2.30258512e+00
  2.50028988e-05 -2.30258512e+00]
Term 'doug' score: [1.73299283e-01 3.46579850e-01 2.50028988e-05 0.00000000e+00
 2.50028988e-05 2.50028988e-05]
Scores now: [1.73299283e-01 3.46579850e-01 2.50028988e-05 0.00000000e+00
 2.50028988e-05 2.50028988e-05]
Field msg_tokenized, Term 'complaint' score: [ 3.33371985e-05 -3.07011350e+00 -3.07011350e+00 -3.07011350e+00
  3.33371985e-05 -3.07011350e+00]
Field name_tokenized, Term 'complaint' score: [-9.2103405 -9.2103405 -9.2103405 -9.2103405 -9.2103405 -9.2103405]
Term 'complaint' score: [3.33371985e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.33371985e-05 0.00000000e+00]
Scores now: [3.33371985e-05 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.33371985e-05 0.00000000e+00]


,name,msg,msg_tokenized,name_tokenized,scores
1,Tom,"\n Doug, we see you're having an issue with...","Terms({'doug', 'wise', 'an', 'what', 'talk', '...",Terms({'tom'}),0.346580
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'is', 'a', 'day', 'not', 'doug', ...",Terms({'doug'}),0.173333
4,Doug,I have complaints about the ski conditions in ...,"Terms({'in', 'west', 'about', 'condit', 'virgi...",Terms({'doug'}),0.000058
2,Doug,"Tom, can I speak to your manager?","Terms({'your', 'can', 'tom', 'to', 'speak', 'i...",Terms({'doug'}),0.000025
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'what', 'doug', 'see', 'do', 'we', 'tha...",Terms({'sue'}),0.000025
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'hi', 'what', 'is', 'boss', 'sue', 'do'...",Terms({'sue'}),0.000000


## Problem - field length

It's a bit better, but another bias that early IR researchers realized happens in text: a single term matching in shorter text matters more than longer text. A tweet mentioning a word once _is very much about that word_ OTOH a book mentioning a word once _is very much NOT about that word_.

In [6]:
from searcharray.similarity import Similarity

def tf_idf_saturate_by_len(term_freqs: np.ndarray,        # TF array of every doc in the index
                           doc_freqs: np.ndarray,         # Doc freq array of every term (> 1 if a phrase)
                           doc_lens: np.ndarray,          # Every documents length (same shape as TF)
                           avg_doc_lens: int,             # avg doc length of corpus
                           num_docs: int) -> np.ndarray:     # total number of docs in corpus
    # the "+ 1e-4" within the log is to avoid log 0 issue
    tf_idf_sat = np.log(term_freqs + 1e-4) / (doc_freqs + 1) # TAKE LOG
    tf_idf_sat /= (doc_lens + 1)
    return tf_idf_sat

QUERY = "doug complaint"
FIELDS = ["msg_tokenized", "name_tokenized"]
query_tokenized = even_better_tokenize(QUERY)

scores = np.zeros(len(msgs))
for query_token in query_tokenized:
    field_scores = np.zeros(len(msgs))
    for field in FIELDS:
        score = msgs[field].array.score(query_token,
                                        similarity=tf_idf_saturate_by_len) # CHANGED
        print(f"Field {field}, Term '{query_token}' score: {score}")
        field_scores = np.maximum(field_scores, score)
    print(f"Term '{query_token}' score: {field_scores}")
    scores += field_scores
    print(f"Scores now: {field_scores}")


msgs['scores'] = scores
msgs.sort_values('scores', ascending=False)

Field msg_tokenized, Term 'doug' score: [ 9.62773793e-03  1.11799952e-02 -2.87823141e-01 -1.77121933e-01
 -2.09325920e-01  2.27299080e-06]
Field name_tokenized, Term 'doug' score: [ 1.25014494e-05 -1.15129256e+00  1.25014494e-05 -1.15129256e+00
  1.25014494e-05 -1.15129256e+00]
Term 'doug' score: [9.62773793e-03 1.11799952e-02 1.25014494e-05 0.00000000e+00
 1.25014494e-05 2.27299080e-06]
Scores now: [9.62773793e-03 1.11799952e-02 1.25014494e-05 0.00000000e+00
 1.25014494e-05 2.27299080e-06]
Field msg_tokenized, Term 'complaint' score: [ 1.85206658e-06 -9.90359194e-02 -3.83764187e-01 -2.36162577e-01
  3.03065440e-06 -2.79101227e-01]
Field name_tokenized, Term 'complaint' score: [-4.60517025 -4.60517025 -4.60517025 -4.60517025 -4.60517025 -4.60517025]
Term 'complaint' score: [1.85206658e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.03065440e-06 0.00000000e+00]
Scores now: [1.85206658e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
 3.03065440e-06 0.00000000e+00]


,name,msg,msg_tokenized,name_tokenized,scores
1,Tom,"\n Doug, we see you're having an issue with...","Terms({'doug', 'wise', 'an', 'what', 'talk', '...",Terms({'tom'}),0.011180
0,Doug,"Hi this is Doug, I have a complaint about the ...","Terms({'hi', 'is', 'a', 'day', 'not', 'doug', ...",Terms({'doug'}),0.009630
4,Doug,I have complaints about the ski conditions in ...,"Terms({'in', 'west', 'about', 'condit', 'virgi...",Terms({'doug'}),0.000016
2,Doug,"Tom, can I speak to your manager?","Terms({'your', 'can', 'tom', 'to', 'speak', 'i...",Terms({'doug'}),0.000013
5,Sue,"Oh doug thats terrible, lets see what we can do.","Terms({'what', 'doug', 'see', 'do', 'we', 'tha...",Terms({'sue'}),0.000002
3,Sue,"Hi, this is Sue, Tom's boss. What can I do for...","Terms({'hi', 'what', 'is', 'boss', 'sue', 'do'...",Terms({'sue'}),0.000000


## BM25 - tuned TF\*IDF to the gills

BM25 (Best Match 25, as in 25th iteration) is the result of many iterations on using these statistics to score relevance.

It's innovation: it combines term frequency + doc length accordingly:

```
tf = term_freqs / (term_freqs + k1 * (1 - b + b * doc_lens / avg_doc_lens))
```

With two constants:

* k1 - controls how fast term freq saturation occurs
* b - ranged 0 to 1, controls how much longer field length will reduce the score

You can construct a raw similarity and play with it here, but its easier to [look at graphs](https://www.desmos.com/calculator/lukbszx5oe) to see the impact of these factors.

## Breadcrumbs for Elasticsearch, Vespa etc

BM25 is the default scoring function in Elasticsearch and Vespa